# Old NF2 code (before lightning)

In [ ]:
#| default_exp pinn_nf2_old

In [ ]:
#| export
import os
import glob
import logging
from tqdm import tqdm
from datetime import datetime

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

import torch
from torch import nn
from torch.utils.data import Dataset
from torch.cuda import get_device_name
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.data import DataLoader, RandomSampler, TensorDataset

from astropy.nddata import block_reduce, block_replicate
from sunpy.map import Map

In [ ]:
#| export
class PotentialModel(nn.Module):

    def __init__(self, b_n, r_p):
        super().__init__()
        self.register_buffer('b_n', b_n)
        self.register_buffer('r_p', r_p)
        c = np.zeros((1, 3))
        c[:, 2] = (1 / np.sqrt(2 * np.pi))
        c = torch.tensor(c, dtype=torch.float32, )
        self.register_buffer('c', c)

    def forward(self, coord):
        v1 = self.b_n[:, None]
        v2 = 2 * np.pi * ((-self.r_p[:, None] + coord[None, :] + self.c[None]) ** 2).sum(-1) ** 0.5
        potential = torch.sum(v1 / v2, dim=0)
        return potential


def get_potential(b_n, height, batch_size=2048, strides=(1, 1, 1), progress=True):
    cube_shape = (*b_n.shape, height)
    strides = (strides, strides, strides) if isinstance(strides, int) else strides
    b_n = b_n.reshape((-1,)).astype(np.float32)
    coords = np.stack(np.mgrid[:cube_shape[0]:strides[0], :cube_shape[1]:strides[1], :cube_shape[2]:strides[2]], -1).reshape((-1, 3))
    r_p = np.stack(np.mgrid[:cube_shape[0], :cube_shape[1], :1], -1).reshape((-1, 3))

    # torch code
    # r = (x * y, 3); coords = (x*y*z, 3), c = (1, 3)
    # --> (x * y, x * y * z, 3) --> (x * y, x * y * z) --> (x * y * z)
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    with torch.no_grad():
        b_n = torch.tensor(b_n, dtype=torch.float32, )
        r_p = torch.tensor(r_p, dtype=torch.float32, )
        model = nn.DataParallel(PotentialModel(b_n, r_p)).to(device)

        coords = torch.tensor(coords, dtype=torch.float32)
        potential = []
        loader = DataLoader(TensorDataset(coords), batch_size=batch_size, num_workers=8)
        it = tqdm(loader, desc='Potential Field') if progress else loader
        for coord, in it:
            coord = coord.to(device)
            p_batch = model(coord)
            potential += [p_batch]

    potential = torch.cat(potential).view(cube_shape).cpu().numpy()
    if strides != (1, 1, 1):
        potential = block_replicate(potential, strides, conserve_sum=False)
    return potential


def get_potential_boundary(b_n, height, batch_size=2048):
    assert not np.any(np.isnan(b_n)), 'Invalid data value'

    cube_shape = (*b_n.shape, height)

    b_n = b_n.reshape((-1)).astype(np.float32)
    coords = [np.stack(np.mgrid[:cube_shape[0], :cube_shape[1], cube_shape[2] - 2:cube_shape[2] + 1], -1),
              np.stack(np.mgrid[:cube_shape[0], -1:2, :cube_shape[2]], -1),
              np.stack(np.mgrid[:cube_shape[0], cube_shape[1] - 2:cube_shape[1] + 1, :cube_shape[2]], -1),
              np.stack(np.mgrid[-1:2, :cube_shape[1], :cube_shape[2]], -1),
              np.stack(np.mgrid[cube_shape[0] - 2:cube_shape[0] + 1, :cube_shape[1], :cube_shape[2]], -1), ]
    coords_shape = [c.shape[:-1] for c in coords]
    flat_coords = np.concatenate([c.reshape(((-1, 3))) for c in coords])

    r_p = np.stack(np.mgrid[:cube_shape[0], :cube_shape[1], :1], -1).reshape((-1, 3))

    # torch code
    # r = (x * y, 3); coords = (x*y*z, 3), c = (1, 3)
    # --> (x * y, x * y * z, 3) --> (x * y, x * y * z) --> (x * y * z)
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    with torch.no_grad():
        b_n = torch.tensor(b_n, dtype=torch.float32, )
        r_p = torch.tensor(r_p, dtype=torch.float32, )
        model = nn.DataParallel(PotentialModel(b_n, r_p, )).to(device)

        flat_coords = torch.tensor(flat_coords, dtype=torch.float32, )

        potential = []
        for coord, in tqdm(DataLoader(TensorDataset(flat_coords), batch_size=batch_size, num_workers=2),
                           desc='Potential Boundary'):
            coord = coord.to(device)
            p_batch = model(coord)
            potential += [p_batch.cpu()]

    potential = torch.cat(potential).numpy()
    idx = 0
    fields = []
    for s in coords_shape:
        p = potential[idx:idx + np.prod(s)].reshape(s)
        b = - 1 * np.stack(np.gradient(p, axis=[0, 1, 2], edge_order=2), axis=-1)
        fields += [b]
        idx += np.prod(s)

    fields = [fields[0][:, :, 1].reshape((-1, 3)),
              fields[1][:, 1, :].reshape((-1, 3)), fields[2][:, 1, :].reshape((-1, 3)),
              fields[3][1, :, :].reshape((-1, 3)), fields[4][1, :, :].reshape((-1, 3))]
    coords = [coords[0][:, :, 1].reshape((-1, 3)),
              coords[1][:, 1, :].reshape((-1, 3)), coords[2][:, 1, :].reshape((-1, 3)),
              coords[3][1, :, :].reshape((-1, 3)), coords[4][1, :, :].reshape((-1, 3))]
    return np.concatenate(coords), np.concatenate(fields)

In [ ]:
#| export
def prep_b_data(b_bottom,
                height, spatial_norm, b_norm,
                potential_boundary=True, potential_strides=4,
                plot=False, plot_path=None):
    # load coordinates
    mf_coords = np.stack(np.mgrid[:b_bottom.shape[0], :b_bottom.shape[1], :1], -1)
    # flatten data
    mf_coords = mf_coords.reshape((-1, 3))
    mf_values = b_bottom.reshape((-1, 3))
    # mf_err = error_cube.reshape((-1, 3))
    # load potential field
    if potential_boundary:
        pf_coords, pf_values = _load_potential_field_data(b_bottom, height, potential_strides)
        # concatenate pf data points
        coords = np.concatenate([pf_coords, mf_coords])
        values = np.concatenate([pf_values, mf_values])
        # err = np.concatenate([pf_err, mf_err])
    else:
        coords = mf_coords
        values = mf_values
        # err = mf_err

    coords = coords.astype(np.float32)
    values = values.astype(np.float32)
    # err = err.astype(np.float32)

    # normalize data
    values = Normalize(-b_norm, b_norm, clip=False)(values) * 2 - 1
    # err = Normalize(0, b_norm, clip=False)(err)

    # apply spatial normalization
    coords = coords / spatial_norm

    # stack to numpy array
    data = np.stack([coords, values], 1)

    if plot:
        _plot_data(b_bottom, plot_path, b_norm)

    return data


def load_hmi_data(data_path):
    if isinstance(data_path, str):
        hmi_p = sorted(glob.glob(os.path.join(data_path, '*Bp.fits')))[0]  # x
        hmi_t = sorted(glob.glob(os.path.join(data_path, '*Bt.fits')))[0]  # y
        hmi_r = sorted(glob.glob(os.path.join(data_path, '*Br.fits')))[0]  # z
        # err_p = sorted(glob.glob(os.path.join(data_path, '*Bp_err.fits')))[0]  # x
        # err_t = sorted(glob.glob(os.path.join(data_path, '*Bt_err.fits')))[0]  # y
        # err_r = sorted(glob.glob(os.path.join(data_path, '*Br_err.fits')))[0]  # z
    else:
        hmi_p, err_p, hmi_r, err_r, hmi_t, err_t = data_path
    # laod maps
    b_bottom = np.stack([Map(hmi_p).data, -Map(hmi_t).data, Map(hmi_r).data]).transpose()
    # error_cube = np.stack([Map(err_p).data, Map(err_t).data, Map(err_r).data]).transpose()
    return b_bottom, Map(hmi_r).meta


def _load_potential_field_data(b_bottom, height, reduce):
    if reduce > 1:
        b_bottom = block_reduce(b_bottom, (reduce, reduce, 1), func=np.mean)
        height = height // reduce
    pf_batch_size = int(1024 * 512 ** 2 / np.prod(b_bottom.shape[:2]))  # adjust batch to AR size
    pf_coords, pf_values = get_potential_boundary(b_bottom[:, :, 2], height, batch_size=pf_batch_size)
    pf_values = np.array(pf_values, dtype=np.float32)
    pf_coords = np.array(pf_coords, dtype=np.float32) * reduce # expand to original coordinate spacing
    # pf_err = np.zeros_like(pf_values)
    return pf_coords, pf_values


def _plot_data(n_b_bottom, plot_path, b_norm):
    fig, axs = plt.subplots(1, 3, figsize=(12, 4))
    axs[0].imshow(n_b_bottom[..., 0].transpose(), vmin=-b_norm, vmax=b_norm, cmap='gray', origin='lower')
    axs[1].imshow(n_b_bottom[..., 1].transpose(), vmin=-b_norm, vmax=b_norm, cmap='gray', origin='lower')
    axs[2].imshow(n_b_bottom[..., 2].transpose(), vmin=-b_norm, vmax=b_norm, cmap='gray', origin='lower')
    plt.savefig(os.path.join(plot_path, 'b.jpg'))
    plt.close()
    # fig, axs = plt.subplots(1, 3, figsize=(12, 4))
    # axs[0].imshow(error_cube[..., 0].transpose(), vmin=0, cmap='gray', origin='lower')
    # axs[1].imshow(error_cube[..., 1].transpose(), vmin=0, cmap='gray', origin='lower')
    # axs[2].imshow(error_cube[..., 2].transpose(), vmin=0, cmap='gray', origin='lower')
    # plt.savefig(os.path.join(plot_path, 'b_err.jpg'))
    # plt.close()


class RandomCoordinateSampler():

    def __init__(self, cube_shape, spatial_norm, batch_size, cuda=True):
        self.cube_shape = cube_shape
        self.spatial_norm = spatial_norm
        self.batch_size = batch_size
        self.float_tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

    def load_sample(self):
        random_coords = self.float_tensor(self.batch_size, 3).uniform_()
        random_coords[:, 0] *= self.cube_shape[0] / self.spatial_norm
        random_coords[:, 1] *= self.cube_shape[1] / self.spatial_norm
        random_coords[:, 2] *= self.cube_shape[2] / self.spatial_norm
        return random_coords

In [ ]:
#| export
class BoundaryDataset(Dataset):

    def __init__(self, batches_path):
        """Data set for lazy loading a pre-batched numpy data array.

        :param batches_path: path to the numpy array.
        """
        self.batches_path = batches_path

    def __len__(self):
        return np.load(self.batches_path, mmap_mode='r').shape[0]

    def __getitem__(self, idx):
        # lazy load data
        d = np.load(self.batches_path, mmap_mode='r')[idx]
        d = np.copy(d)
        coord, field = d[:, 0],  d[:, 1]
        return coord, field

class ImageDataset(Dataset):

    def __init__(self, cube_shape, norm, z=0):
        coordinates = np.stack(np.mgrid[:cube_shape[0],
                               :cube_shape[1]], -1)
        self.coordinates = coordinates
        self.coordinates_flat = coordinates.reshape((-1, 2))
        self.norm = norm
        self.z = z / self.norm

    def __len__(self, ):
        return self.coordinates_flat.shape[0]

    def __getitem__(self, idx):
        coord = self.coordinates_flat[idx]
        scaled_coord = [coord[0] / self.norm,
                        coord[1] / self.norm,
                        self.z]
        return np.array(scaled_coord, dtype=np.float32)


class CubeDataset(Dataset):

    def __init__(self, cube_shape, norm, block_shape=(32, 32, 32), coords=[], strides=1):
        self.cube_shape = cube_shape
        self.block_shape = block_shape
        self.coords = coords
        self.strides = strides
        self.norm = norm

    def __len__(self):
        return len(self.coords)

    def __getitem__(self, idx):
        coord = self.coords[idx]
        return self.getCube(coord)

    def getCube(self, coord):
        coord_cube = np.stack(np.mgrid[coord[0]:coord[0] + self.block_shape[0]:self.strides,
                              coord[1]:coord[1] + self.block_shape[1]:self.strides,
                              coord[2]:coord[2] + self.block_shape[2]:self.strides, ], -1)
        coord_cube = np.stack([coord_cube[..., 0] / self.norm,
                               coord_cube[..., 1] / self.norm,
                               coord_cube[..., 2] / self.norm, ], -1)
        return coord_cube.astype(np.float32)

In [ ]:
#| export
class Sine(nn.Module):
    def __init__(self, w0=1.):
        super().__init__()
        self.w0 = w0

    def forward(self, x):
        return torch.sin(self.w0 * x)


class BModel(nn.Module):

    def __init__(self, in_coords, out_values, dim, pos_encoding=False):
        super().__init__()
        if pos_encoding:
            posenc = PositionalEncoding(8, 20)
            d_in = nn.Linear(in_coords * 40, dim)
            self.d_in = nn.Sequential(posenc, d_in)
        else:
            self.d_in = nn.Linear(in_coords, dim)
        lin = [nn.Linear(dim, dim) for _ in range(8)]
        self.linear_layers = nn.ModuleList(lin)
        self.d_out = nn.Linear(dim, out_values)
        self.activation = Sine()  # torch.tanh

    def forward(self, x):
        x = self.activation(self.d_in(x))
        for l in self.linear_layers:
            x = self.activation(l(x))
        x = self.d_out(x)
        return x

class VectorPotentialModel(nn.Module):

    def __init__(self, in_coords, dim, pos_encoding=False):
        super().__init__()
        if pos_encoding:
            posenc = PositionalEncoding(8, 20)
            d_in = nn.Linear(in_coords * 40, dim)
            self.d_in = nn.Sequential(posenc, d_in)
        else:
            self.d_in = nn.Linear(in_coords, dim)
        lin = [nn.Linear(dim, dim) for _ in range(8)]
        self.linear_layers = nn.ModuleList(lin)
        self.d_out = nn.Linear(dim, 3)
        self.activation = Sine()  # torch.tanh

    def forward(self, x):
        coord = x
        x = self.activation(self.d_in(x))
        for l in self.linear_layers:
            x = self.activation(l(x))
        a = self.d_out(x)
        #
        jac_matrix = jacobian(a, coord)
        dAy_dx = jac_matrix[:, 1, 0]
        dAz_dx = jac_matrix[:, 2, 0]
        dAx_dy = jac_matrix[:, 0, 1]
        dAz_dy = jac_matrix[:, 2, 1]
        dAx_dz = jac_matrix[:, 0, 2]
        dAy_dz = jac_matrix[:, 1, 2]
        rot_x = dAz_dy - dAy_dz
        rot_y = dAx_dz - dAz_dx
        rot_z = dAy_dx - dAx_dy
        b = torch.stack([rot_x, rot_y, rot_z], -1)
        #
        return b

class PositionalEncoding(nn.Module):
    """
    Positional Encoding of the input coordinates.

    encodes x to (..., sin(2^k x), cos(2^k x), ...)
    k takes "num_freqs" number of values equally spaced between [0, max_freq]
    """

    def __init__(self, max_freq, num_freqs):
        """
        Args:
            max_freq (int): maximum frequency in the positional encoding.
            num_freqs (int): number of frequencies between [0, max_freq]
        """
        super().__init__()
        freqs = 2 ** torch.linspace(0, max_freq, num_freqs)
        self.register_buffer("freqs", freqs)  # (num_freqs)

    def forward(self, x):
        """
        Inputs:
            x: (batch, num_samples, in_features)
        Outputs:
            out: (batch, num_samples, 2*num_freqs*in_features)
        """
        x_proj = x.unsqueeze(dim=-2) * self.freqs.unsqueeze(dim=-1)  # (num_rays, num_samples, num_freqs, in_features)
        x_proj = x_proj.reshape(*x.shape[:-1], -1)  # (num_rays, num_samples, num_freqs*in_features)
        out = torch.cat([torch.sin(x_proj), torch.cos(x_proj)],
                        dim=-1)  # (num_rays, num_samples, 2*num_freqs*in_features)
        return out


def jacobian(output, coords):
    jac_matrix = [torch.autograd.grad(output[:, i], coords,
                                      grad_outputs=torch.ones_like(output[:, i]).to(output),
                                      retain_graph=True,
                                      create_graph=True)[0]
                  for i in range(output.shape[1])]
    jac_matrix = torch.stack(jac_matrix, dim=1)
    return jac_matrix

In [ ]:
#| export
class NF2Trainer:

    def __init__(self, base_path, b_bottom, height, spatial_norm, b_norm, meta_info=None, dim=256,
                 positional_encoding=False, meta_path=None, use_potential_boundary=True, potential_strides=4, use_vector_potential=False,
                 w_div=0.1, w_ff=0.1, decay_iterations=None,
                 device=None, work_directory=None):
        """Magnetic field extrapolations trainer

        :param base_path: path to the results folder.
        :param b_bottom: magnetic field data (x, y, (Bp, -Bt, Br)).
        :param error_cube: associated error information.
        :param height: height of simulation volume.
        :param spatial_norm: normalization of coordinate axis.
        :param b_norm: normalization of magnetic field strength.
        :param meta_info: additional data information. stored in the save state.
        :param dim: number of neurons per layer (8 layers).
        :param positional_encoding: use positional encoding.
        :param meta_path: start from a pre-learned simulation state.
        :param use_potential_boundary: use potential field as boundary condition. If None use an open boundary.
        :param potential_strides: use binned potential field boundary condition. Only applies if use_potential_boundary = True.
        :param use_vector_potential: derive the magnetic field from a vector potential.
        :param w_div: weighting parameter for divergence freeness of the simulation.
        :param w_ff: weighting parameter for force freeness of the simulation.
        :param decay_iterations: decay weighting for boundary condition (w_bc=1000) over n iterations to 1.
        :param device: device for model training.
        :param work_directory: directory to store scratch data (prepared batches).
        """

        # general parameters
        self.base_path = base_path
        work_directory = base_path if work_directory is None else work_directory
        self.work_directory = work_directory
        # self.save_path = os.path.join(base_path, 'extrapolation_result.nf2')
        self.checkpoint_path = os.path.join(base_path, 'checkpoint.pt')

        # data parameters
        self.spatial_norm = spatial_norm
        self.height = height
        self.b_norm = b_norm
        self.meta_info = meta_info

        # init directories
        os.makedirs(base_path, exist_ok=True)
        os.makedirs(work_directory, exist_ok=True)

        # init logging
        logger = logging.getLogger()
        logger.setLevel(logging.INFO)
        for hdlr in logger.handlers[:]:  # remove all old handlers
            logger.removeHandler(hdlr)
        logger.addHandler(logging.FileHandler("{0}/{1}.log".format(base_path, "info_log")))  # set the new file handler
        logger.addHandler(logging.StreamHandler())  # set the new console handler

        self.logger = logger

        # log settings
        self.logger.info('Configuration:')
        self.logger.info(
            'dim: %d, w_div: %f, w_ff: %f, decay_iterations: %s, potential: %s, vector_potential: %s, ' % (
                dim, w_div, w_ff, str(decay_iterations), str(use_potential_boundary),
                str(use_vector_potential)))

        # # setup device
        n_gpus = torch.cuda.device_count()
        device_names = [get_device_name(i) for i in range(n_gpus)]
        device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        self.logger.info('Using device: %s (gpus %d) %s' % (str(device), n_gpus, str(device_names)))
        self.device = device

        # prepare data
        self.b_bottom = b_bottom
        # self.error_cube = error_cube

        # load dataset
        self.data = prep_b_data(b_bottom, height, spatial_norm, b_norm,
                                plot=True, plot_path=base_path,
                                potential_boundary=use_potential_boundary, potential_strides=potential_strides)
        self.cube_shape = [*b_bottom.shape[:-1], height]

        # init model
        # if use_vector_potential:
        #     model = VectorPotentialModel(3, dim, pos_encoding=positional_encoding)
        # else:
        self.model = nn.DataParallel(BModel(3, 3, dim, pos_encoding=positional_encoding)).to(device)
        self.opt = torch.optim.Adam(self.model.parameters(), lr=5e-4)

        # load last state
        if os.path.exists(self.checkpoint_path):
            state_dict = torch.load(self.checkpoint_path, map_location=device)
            self.model.load_state_dict(state_dict['m'])
            self.opt.load_state_dict(state_dict['o'])

            start_iteration = state_dict['iteration']
            self.w_bc = state_dict['w_bc']
            self.logger.info('Resuming training from iteration %d' % start_iteration)
        else:
            if meta_path:
                state_dict = torch.load(meta_path, map_location=device)['m']
                self.model.load_state_dict(state_dict)
                self.opt = torch.optim.Adam(self.model.parameters(), lr=5e-5)
                self.logger.info('Loaded meta state: %s' % meta_path)

            # init
            start_iteration = 0
            self.w_bc = 1000 if decay_iterations else 1


        self.start_iteration = start_iteration
        self.w_bc_decay = (1 / 1000) ** (1 / decay_iterations) if decay_iterations is not None else 1
        self.w_div, self.w_ff = w_div, w_ff

    def train(self, total_iterations, lr_decay_iterations, co_batch_size, bc_batch_size, log_interval=100, validation_interval=100, num_workers=None):
        """Start magnetic field extrapolation fit.

        :param total_iterations: number of iterations for training.
        :param batch_size: number of samples per iteration.
        :param log_interval: log training details every nth iteration.
        :param validation_interval: evaluate simulation every nth iteration.
        :param num_workers: number of workers for data loading (default system spec).
        :return: path of the final save state.
        """
        start_time = datetime.now()
        self.log_interval = log_interval
        num_workers = os.cpu_count() if num_workers is None else num_workers

        # model = self.parallel_model
        opt = self.opt
        # history = self.history
        device = self.device

        # init
        sampler = RandomCoordinateSampler(self.cube_shape, self.spatial_norm, co_batch_size)
        self.scheduler = ExponentialLR(opt, gamma=(5e-5 / 5e-4) ** (1 / lr_decay_iterations))
        # iterations = total_iterations - self.start_iteration
        self.total_iterations = total_iterations
        iterations = self.total_iterations - self.start_iteration
        if iterations <= 0:
            print('Training already finished!')
            return

        # init loader
        data_loader, batches_path = self._init_loader(bc_batch_size, self.data, num_workers, iterations)

        losses = []
        losses_no_weight = []

        # total_loss_bc = []
        # total_loss_div = []
        # total_loss_ff = []
        self.model.train()
        for iter, (boundary_coords, b_true) in tqdm(enumerate(data_loader, start=self.start_iteration),
                                                           total=len(data_loader), desc='Training'):
            opt.zero_grad()
            # load input data
            boundary_coords, b_true = boundary_coords.to(device), b_true.to(device)
            random_coords = sampler.load_sample()

            # concatenate boundary and random points
            n_boundary_coords = boundary_coords.shape[0]
            coords = torch.cat([boundary_coords, random_coords], 0)
            coords.requires_grad = True

            # forward step
            b = self.model(coords)

            # compute boundary loss
            boundary_b = b[:n_boundary_coords]
            loss_bc = torch.clip(torch.abs(boundary_b - b_true), 0)
            self.loss_bc = torch.mean(loss_bc.pow(2).sum(-1))

            # compute div and ff loss
            loss_div, loss_ff = calculate_loss(b, coords)
            self.loss_div, self.loss_ff = loss_div.mean(), loss_ff.mean()

            # reset grad from auto-gradient operation
            opt.zero_grad()
            # compute loss
            self.loss = (self.loss_bc * self.w_bc + 
                         self.loss_div * self.w_div + 
                         self.loss_ff * self.w_ff)
            losses.append(self.loss.detach().cpu().numpy())
            loss_no_weight = self.loss_bc + self.loss_ff + self.loss_div
            losses_no_weight.append(loss_no_weight.detach().cpu().numpy())
            
            if iter == 0:
                self.print_log(iter)
                self.save(iter)

            self.loss.backward()
            # update step
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 0.1)
            opt.step()

            # save loss information
            # total_loss_bc += [loss_bc.detach().cpu().numpy()]
            # total_loss_div += [loss_div.mean().detach().cpu().numpy()]
            # total_loss_ff += [loss_ff.mean().detach().cpu().numpy()]

            if (self.log_interval > 0 and (iter + 1) % self.log_interval == 0):
                self.print_log(iter+1)
                self.save(iter+1)
                self.save_state_dict(iter+1, 'checkpoint.pt')

            # update training parameters
            if self.w_bc > 1:
                self.w_bc *= self.w_bc_decay
                if self.w_bc <= 1:
                    self.w_bc = 1
            if self.scheduler.get_last_lr()[0] > 5e-5:
                self.scheduler.step()
            # logging
            # if log_interval > 0 and (iter + 1) % log_interval == 0:
            #     # log loss
            #     logging.info('[Iteration %06d/%06d] [B-Field: %.08f; Div: %.08f; For: %.08f] [%s]' %
            #                  (iter + 1, iterations,
            #                   np.mean(total_loss_bc),
            #                   np.mean(total_loss_div),
            #                   np.mean(total_loss_ff),
            #                   datetime.now() - start_time))
            #     # reset
            #     total_loss_bc = []
            #     total_loss_div = []
            #     total_loss_ff = []

            #     # plot sample
            #     model.eval()
            #     self.plot_sample(iter, batch_size=batch_size)
            #     model.train()

            #     # log decay parameters
            #     logging.info('Lambda B: %f' % (self.w_bc))
            #     logging.info('LR: %f' % (scheduler.get_last_lr()[0]))
            # # validation
            # if validation_interval > 0 and (iter + 1) % validation_interval == 0:
            #     model.eval()
            #     self.save(iter)
            #     # validate and plot
            #     mean_b, total_divergence, mean_force, sigma_angle = self.validate(self.height, batch_size)
            #     logging.info('Validation [Cube: B: %.03f; Div: %.03f; For: %.03f; Sig: %.03f]' %
            #                  (mean_b, total_divergence, mean_force, sigma_angle))
            #     #
            #     history['iteration'].append(iter + 1)
            #     history['b_loss'].append(mean_b.mean())
            #     history['loss_div'].append(total_divergence)
            #     history['loss_ff'].append(mean_force)
            #     history['sigma_angle'].append(sigma_angle)
            #     self.plotHistory()
            #     #
            #     model.train()
        # save final model state
        # torch.save({'m': self.model.state_dict(),
        #             'o': self.opt.state_dict(), },
        #            os.path.join(self.base_path, 'final.pt'))
        # torch.save({'model': self.model,
        #             'cube_shape': self.cube_shape,
        #             'b_norm': self.b_norm,
        #             'spatial_norm': self.spatial_norm,
        #             'meta_info': self.meta_info}, self.save_path)
        # cleanup

        np.save(os.path.join(self.base_path, 'losses.npy'), np.array(losses))
        np.save(os.path.join(self.base_path, 'losses_no_weight.npy'), np.array(losses_no_weight))
        self.print_log(self.total_iterations)
        self.save(self.total_iterations)
        self.save(self.total_iterations, 'model_final.pt')
        self.save_state_dict(self.total_iterations, 'checkpoint_final.pt')
        os.remove(batches_path)

        # return self.save_path

    def _init_loader(self, batch_size, data, num_workers, iterations):
        # shuffle data
        r = np.random.permutation(data.shape[0])
        data = data[r]
        # adjust to batch size
        pad = batch_size - data.shape[0] % batch_size
        data = np.concatenate([data, data[:pad]])
        # split data into batches
        n_batches = data.shape[0] // batch_size
        batches = np.array(np.split(data, n_batches), dtype=np.float32)
        # store batches to disk
        batches_path = os.path.join(self.work_directory, 'batches.npy')
        np.save(batches_path, batches)
        # create data loaders
        dataset = BoundaryDataset(batches_path)
        # create loader
        data_loader = DataLoader(dataset, batch_size=None, num_workers=num_workers, pin_memory=True,
                                 sampler=RandomSampler(dataset, replacement=True, num_samples=iterations))
        return data_loader, batches_path

    # def save(self, iteration):
    #     torch.save({'model': self.model,
    #                 'cube_shape': self.cube_shape,
    #                 'b_norm': self.b_norm,
    #                 'spatial_norm': self.spatial_norm,
    #                 'meta_info': self.meta_info}, self.save_path)
    #     torch.save({'iteration': iteration + 1,
    #                 'm': self.model.state_dict(),
    #                 'o': self.opt.state_dict(),
    #                 'history': self.history,
    #                 'w_bc': self.w_bc},
    #                self.checkpoint_path)
        
    def save(self, iteration, filename=None):
        if filename is None:
            filename = 'model_%06d.pt' % iteration

        torch.save({'iteration': iteration,
                    'model': self.model,
                    'cube_shape': self.cube_shape,
                    'b_norm': self.b_norm,
                    'spatial_norm': self.spatial_norm,
                    'loss_bc': self.loss_bc.detach().cpu().numpy(),
                    'w_bc': self.w_bc,
                    'loss_div': self.loss_div.detach().cpu().numpy(),
                    'w_div': self.w_div,
                    'loss_ff': self.loss_ff.detach().cpu().numpy(),
                    'w_ff': self.w_ff,
                    'LR':self.scheduler.get_last_lr()[0]}, 
                    os.path.join(self.base_path, filename))
        
    def save_state_dict(self, iteration, filename):
        torch.save({'iteration': iteration,
                    'w_bc': self.w_bc,
                    'm': self.model.state_dict(),
                    'o': self.opt.state_dict()},
                    os.path.join(self.base_path, filename))

    def print_log(self, iteration):
        self.logger.info('[Iteration %06d/%06d] [loss: %.08f] [loss_bc: %.08f; loss_div: %.08f; loss_ff: %.08f] [w_bc: %f, LR: %f]' %
                (iteration, self.total_iterations,
                self.loss,
                self.w_bc*self.loss_bc,
                self.w_ff*self.loss_ff,
                self.w_div*self.loss_div,
                self.w_bc,
                self.scheduler.get_last_lr()[0]))

    def plot_sample(self, iteration, n_samples=10, batch_size=4096):
        fig, axs = plt.subplots(3, n_samples, figsize=(n_samples * 4, 12))
        heights = np.linspace(0, 1, n_samples) ** 2 * (self.height - 1)  # more samples from lower heights
        imgs = np.array([self.get_image(h, batch_size) for h in heights])
        for i in range(3):
            for j in range(10):
                v_min_max = np.max(np.abs(imgs[j, ..., i]))
                axs[i, j].imshow(imgs[j, ..., i].transpose(), cmap='gray', vmin=-v_min_max, vmax=v_min_max,
                                 origin='lower')
                axs[i, j].set_axis_off()
        for j, h in enumerate(heights):
            axs[0, j].set_title('%.01f' % h)
        fig.tight_layout()
        fig.savefig(os.path.join(self.base_path, '%06d.jpg' % (iteration + 1)))
        plt.close(fig)

    def get_image(self, z=0, batch_size=4096):
        image_loader = DataLoader(ImageDataset([*self.cube_shape, 3], self.spatial_norm, z),
                                  batch_size=batch_size, shuffle=False)
        image = []
        for coord in image_loader:
            coord.requires_grad = True
            pred_pix = self.model(coord.to(self.device))
            image.extend(pred_pix.detach().cpu().numpy())
        image = np.array(image).reshape((*self.cube_shape[:2], 3))
        return image

    def validate(self, z, batch_size):
        b, j, div, coords = self.get_cube(z, batch_size)
        b = b.unsqueeze(0) * self.b_norm
        j = j.unsqueeze(0) * self.b_norm / self.spatial_norm
        div = div.unsqueeze(0) * self.b_norm / self.spatial_norm

        norm = b.pow(2).sum(-1).pow(0.5) * j.pow(2).sum(-1).pow(0.5)
        angle = torch.cross(j, b, dim=-1).pow(2).sum(-1).pow(0.5) / norm
        sig = torch.asin(torch.clip(angle, -1. + 1e-7, 1. - 1e-7)) * (180 / np.pi)
        sig = torch.abs(sig)
        weighted_sig = np.average(sig.numpy(), weights=j.pow(2).sum(-1).pow(0.5).numpy())

        loss_bc = torch.abs(b[0, :, :, 0, :] - self.b_bottom)
        loss_bc = torch.clip(loss_bc, 0, None)
        loss_bc = torch.sqrt((loss_bc ** 2).sum(-1))

        b_norm = b.pow(2).sum(-1).pow(0.5) + 1e-7
        div_loss = div / b_norm
        for_loss = torch.cross(j, b, dim=-1).pow(2).sum(-1).pow(0.5) / b_norm

        return loss_bc.mean().numpy(), torch.mean(div_loss).numpy(), \
               torch.mean(for_loss).numpy(), weighted_sig

    def get_cube(self, max_height, batch_size=int(1e4)):
        b = []
        j = []
        div = []

        coords = np.stack(np.mgrid[:self.cube_shape[0], :self.cube_shape[1], :max_height], -1)
        coords = torch.tensor(coords / self.spatial_norm, dtype=torch.float32)
        coords_shape = coords.shape[:-1]
        coords = coords.view((-1, 3))
        for k in tqdm(range(int(np.ceil(coords.shape[0] / batch_size))), desc='Validation'):
            coord = coords[k * batch_size: (k + 1) * batch_size]
            coord.requires_grad = True
            coord = coord.to(self.device)
            b_batch = self.model(coord)

            jac_matrix = jacobian(b_batch, coord)
            dBx_dx = jac_matrix[:, 0, 0]
            dBy_dx = jac_matrix[:, 1, 0]
            dBz_dx = jac_matrix[:, 2, 0]
            dBx_dy = jac_matrix[:, 0, 1]
            dBy_dy = jac_matrix[:, 1, 1]
            dBz_dy = jac_matrix[:, 2, 1]
            dBx_dz = jac_matrix[:, 0, 2]
            dBy_dz = jac_matrix[:, 1, 2]
            dBz_dz = jac_matrix[:, 2, 2]
            #
            rot_x = dBz_dy - dBy_dz
            rot_y = dBx_dz - dBz_dx
            rot_z = dBy_dx - dBx_dy
            #
            j_batch = torch.stack([rot_x, rot_y, rot_z], -1)
            div_batch = torch.abs(dBx_dx + dBy_dy + dBz_dz)
            #
            b += [b_batch.detach().cpu()]
            j += [j_batch.detach().cpu()]
            div += [div_batch.detach().cpu()]

        b = torch.cat(b, dim=0).view((*coords_shape, 3))
        j = torch.cat(j, dim=0).view((*coords_shape, 3))
        div = torch.cat(div, dim=0).view(coords_shape)
        return b, j, div, coords

    def plotHistory(self):
        history = self.history
        plt.figure(figsize=(12, 16))
        plt.subplot(411)
        plt.plot(history['iteration'], history['b_loss'], label='B')
        plt.xlabel('Iteration')
        plt.ylabel('B')
        plt.subplot(412)
        plt.plot(history['iteration'], history['loss_div'], label='Divergence')
        plt.xlabel('Iteration')
        plt.ylabel('Divergence')
        plt.subplot(413)
        plt.plot(history['iteration'], history['loss_ff'], label='Force')
        plt.xlabel('Iteration')
        plt.ylabel('Force')
        plt.subplot(414)
        plt.plot(history['iteration'], history['sigma_angle'], label='Angle')
        plt.xlabel('Iteration')
        plt.ylabel('Angle')
        plt.tight_layout()
        plt.savefig(os.path.join(self.base_path, 'history.jpg'))
        plt.close()


def calculate_loss(b, coords):
    jac_matrix = jacobian(b, coords)
    dBx_dx = jac_matrix[:, 0, 0]
    dBy_dx = jac_matrix[:, 1, 0]
    dBz_dx = jac_matrix[:, 2, 0]
    dBx_dy = jac_matrix[:, 0, 1]
    dBy_dy = jac_matrix[:, 1, 1]
    dBz_dy = jac_matrix[:, 2, 1]
    dBx_dz = jac_matrix[:, 0, 2]
    dBy_dz = jac_matrix[:, 1, 2]
    dBz_dz = jac_matrix[:, 2, 2]
    #
    rot_x = dBz_dy - dBy_dz
    rot_y = dBx_dz - dBz_dx
    rot_z = dBy_dx - dBx_dy
    #
    j = torch.stack([rot_x, rot_y, rot_z], -1)
    jxb = torch.cross(j, b, -1)
    loss_ff = torch.sum(jxb ** 2, dim=-1) / (torch.sum(b ** 2, dim=-1) + 1e-7)
    loss_div = (dBx_dx + dBy_dy + dBz_dz) ** 2
    return loss_div, loss_ff